In [1]:
import numpy as np
import pandas as pd
import os
from tabulate import tabulate

In [2]:
class Strace_Stats:
    
    def __init__(self,root):
        self.root = root
        self.dfs = []
    
    def get_filenames(self):
        filenames = os.listdir(self.root)
        return filenames
    
    def get_headings(self):
        headings = []
        substring = " Strace Statistics"
        for files in self.filenames:
            headings.append(files[:-4]+substring)
        return headings
    
    def strace_stats(self):
        self.filenames = self.get_filenames()
        self.headings = self.get_headings()
        for idx in range(len(self.filenames)):
            self.dfs.append(self.get_strace_stats(self.filenames[idx],self.headings[idx]))
    
    def get_strace_stats(self,filename,heading):
        path = os.path.join(self.root,filename)
        file = open(path,"r")
        data = file.read()
        file.close()
        data = data.split('\n')
        processed_data = []
        for i in data:
            string = i.split()
            if '------' not in string:
                processed_data.append(string)

        processed_data = processed_data[:-2]

        for lines in range(1,len(processed_data)):
            if len(processed_data[lines])==5:
                temp = ",".join([str(ele) for ele in processed_data[lines][:-1]])
                temp += ',0,'
                temp += processed_data[lines][-1]
                temp = temp.split(",")
                processed_data[lines] = temp
            for i in range(len(processed_data[lines])-1):
                processed_data[lines][i] = float(processed_data[lines][i])

        df_cols = processed_data[0][1:]
        df = pd.DataFrame(processed_data[1:],columns=df_cols)
        df = df[["time","calls","syscall"]]
        df = df.sort_values(by="calls",ascending=False).reset_index(drop=True)
        df = df[:10]
        return (df,heading)
        
    def print_tables(self):
        for df in self.dfs:
            print("\n",df[1],":")
            print(tabulate(df[0],headers=df[0].columns, tablefmt = 'psql'))

In [3]:
class Lock_Stats:
    def __init__(self,root):
        self.root = root
        self.dfs = []
    
    def get_filenames(self):
        filenames = os.listdir(self.root)
        return filenames
    
    def get_headings(self):
        headings = []
        substring = " Lock Stats" if "Top_Locks" not in self.root else " Top Locks Stats"
        for files in self.filenames:
            headings.append(files[:-4]+substring)
        return headings
    
    def lock_stats(self):
        self.filenames = self.get_filenames()
        self.headings = self.get_headings()
        for idx in range(len(self.filenames)):
            self.dfs.append(self.get_lock_stats(self.filenames[idx],self.headings[idx]))
    
    def get_lock_stats(self,filename,heading):
        path = os.path.join(self.root,filename)
        file = open(path,"r")
        data = file.read().split("\n")
        file.close()

        temp_processed_data = []
        for lines in range(len(data)):
            if "---" not in data[lines] and "..." not in data[lines] and len(data[lines]) != 0:
                temp_processed_data.append(data[lines])

        processed_data = []
        for lines in range(len(temp_processed_data)):
            if "[<00" not in temp_processed_data[lines]:
                processed_data.append(temp_processed_data[lines])
        processed_data = processed_data[1:]

        split_processed_data = []
        for lines in range(len(processed_data)):
            if ", " in processed_data[lines] or "{ " in processed_data[lines] or "+ " in processed_data[lines] or " ?" in processed_data[lines] or " m" in processed_data[lines]:
                corrected_line = []
                lock = processed_data[lines].split(":")
                if len(lock)!=2:
                    lock_name = lock[0]+":"+lock[1]
                    lock = lock[1:]
                    lock[0] = lock_name
                stats = lock[1].split()
                corrected_line.append(lock[0].strip()+":")
                for values in stats:
                    corrected_line.append(values)
                split_processed_data.append(corrected_line)
            else:
                split_processed_data.append(processed_data[lines].split())

        processed_data = split_processed_data
        processed_data[0] = processed_data[0][1:]

        try:
            for lines in range(1,len(processed_data)):
                for values in range(0,len(processed_data[lines])):
                    if values==0:
                        processed_data[lines][0] = processed_data[lines][0][:-1]
                    else:
                        processed_data[lines][values] = float(processed_data[lines][values])
        except:
            print(processed_data[lines])
        
        df_cols =  processed_data[0]
        df_cols[0] = "lock_name"

        try:
            df = pd.DataFrame(data=processed_data[1:], columns=df_cols)
        except Exception as e:
            print(e)
            print(self.filename)
            print(processed_data)
            
        return (df,heading)

In [4]:
StraceStats = Strace_Stats(root="./strace")
LockStats = Lock_Stats(root="./lock_files/Locks")
TopLockStats = Lock_Stats(root="./lock_files/Top_Locks")

In [5]:
StraceStats.strace_stats()
LockStats.lock_stats()
TopLockStats.lock_stats()

In [6]:
StraceStats.print_tables()


 Browser Strace Statistics :
+----+--------+---------+------------+
|    |   time |   calls | syscall    |
|----+--------+---------+------------|
|  0 |  13.28 |     936 | openat     |
|  1 |  11.46 |     647 | mmap       |
|  2 |   6.31 |     582 | mprotect   |
|  3 |   6.76 |     581 | close      |
|  4 |  10.43 |     571 | futex      |
|  5 |   6.48 |     553 | read       |
|  6 |   4.85 |     390 | access     |
|  7 |   4.09 |     290 | newfstatat |
|  8 |   1.95 |     209 | fstatfs    |
|  9 |   2.14 |     202 | stat       |
+----+--------+---------+------------+

 cat Strace Statistics :
+----+--------+---------+------------+
|    |   time |   calls | syscall    |
|----+--------+---------+------------|
|  0 |  26.79 |      10 | mmap       |
|  1 |  13.71 |       6 | close      |
|  2 |   6.81 |       4 | openat     |
|  3 |   4.94 |       4 | pread64    |
|  4 |  10.22 |       3 | mprotect   |
|  5 |   5.69 |       3 | brk        |
|  6 |   5.46 |       3 | read       |
|  7 |  

In [7]:
len(LockStats.dfs)

8

In [8]:
len(TopLockStats.dfs)

8

In [22]:
TopLockStats.dfs[1][0]

,lock_name,con-bounces,contentions,waittime-min,waittime-max,waittime-total,waittime-avg,acq-bounces,acquisitions,holdtime-min,holdtime-max,holdtime-total,holdtime-avg
0,&mm->mmap_lock#2-W,988.0,2028.0,0.13,4144.87,215119.52,106.07,5262.0,34387.0,0.07,6485.22,393091.59,11.43
1,&mm->mmap_lock#2-R,2499.0,6669.0,0.13,4134.46,533993.91,80.07,9190.0,754270.0,0.09,6703.77,4496747.37,5.96
2,&rq->lock,8391.0,8407.0,0.12,485.07,51277.73,6.10,230104.0,1729108.0,0.10,1278.87,5922904.40,3.43
3,&futex_queues[i].lock,4246.0,4251.0,0.07,360.85,2207.77,0.52,115791.0,148187.0,0.05,487.09,100055.48,0.68
4,ptlock_ptr(page)#2,3539.0,3544.0,0.07,173.42,9237.97,2.61,76426.0,896608.0,0.10,1070.14,1522675.91,1.70
5,&ep->lock-W,2066.0,2066.0,0.08,184.35,2398.83,1.16,141559.0,642928.0,0.05,253.03,185098.45,0.29
6,&ep->lock-R,1116.0,1120.0,0.10,247.19,655.13,0.58,213424.0,400067.0,0.06,295.16,1341832.35,3.35
7,&lruvec->lru_lock,2330.0,2334.0,0.12,486.38,5257.12,2.25,32184.0,136014.0,0.10,240.32,283910.94,2.09
8,&u->lock,2107.0,2107.0,0.08,102.54,1157.91,0.55,339963.0,1275199.0,0.07,252.62,460722.16,0.36
9,logbuf_lock,1785.0,1793.0,0.06,29.64,687.23,0.38,3475.0,53098.0,0.07,68.81,17972.74,0.34
